In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import numpy as np

import matplotlib.pyplot as plt
import rasterio
import os
import io
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(filename=".env"), override=True)


True

# Establishing connection with Sentinel 2

In [99]:
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")

# set up credentials
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
                          client_secret=CLIENT_SECRET, include_client_id=True)

# Downloading .tiff images

In [100]:
lago_atitlan = {
"west": -91.326256,
"east": -91.07151,
"south": 14.5948,
"north": 14.750979
}

lago_amatitlan = {
"west": -90.638065,
"east": -90.512924,
"south": 14.412347,
"north": 14.493799
}

bbox_amatitlan = [lago_amatitlan['west'], lago_amatitlan['south'], lago_amatitlan['east'], lago_amatitlan['north']]
bbox_atitlan = [lago_atitlan['west'], lago_atitlan['south'], lago_atitlan['east'], lago_atitlan['north']]

collection_id = "sentinel-2-l2a"

In [ ]:
# ---------------- Evalscript: 9 bandas crudas (sin B29) ----------------
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B02","B03","B04","B05","B07","B08","B8A","B11","B12"],
    output: { bands: 9, sampleType: "FLOAT32" }
  };
}
function evaluatePixel(s) {
  // Orden fijo para luego: 0:B02,1:B03,2:B04,3:B05,4:B07,5:B08,6:B8A,7:B11,8:B12
  return [s.B02, s.B03, s.B04, s.B05, s.B07, s.B08, s.B8A, s.B11, s.B12];
}
"""

# Fechas con nubosidad menor a 20
fechas_bajas_nubosidad = [
    "2025-02-07", "2025-02-10", "2025-02-25", "2025-02-27", "2025-03-02", "2025-03-04",
    "2025-03-07", "2025-03-09", "2025-03-12", "2025-03-14", "2025-03-19", "2025-03-22",
    "2025-03-24", "2025-03-26", "2025-04-03", "2025-04-11", "2025-04-13", "2025-04-15",
    "2025-04-16", "2025-04-18", "2025-04-28", "2025-05-03", "2025-05-13", "2025-05-28",
    "2025-07-10", "2025-07-17", "2025-07-20", "2025-07-24", "2025-08-01"
 ]

# Diccionario de lagos y sus bounding boxes
lagos = {
    'amatitlan': bbox_amatitlan,
    'atitlan': bbox_atitlan
}

# Descargar y guardar un archivo por lago y fecha
os.makedirs("out", exist_ok=True)

for lago, bbox in lagos.items():
    for fecha in fechas_bajas_nubosidad:
        data_entry = [{
            "type": "sentinel-2-l1c",
            "dataFilter": {
                "timeRange": {
                    "from": f"{fecha}T00:00:00Z",
                    "to": f"{fecha}T23:59:59Z"
                },
                "maxCloudCoverage": 20
            }
        }]
        json_request = {
            "input": {
                "bounds": { "bbox": bbox },
                "data": data_entry
            },
            "output": {
                "width": 1024,
                "responses": [{
                    "identifier": "default",
                    "format": {
                        "type": "image/tiff",
                        "parameters": { "compression": "DEFLATE" }
                    }
                }]
            },
            "evalscript": evalscript
        }
        url_request = "https://services.sentinel-hub.com/api/v1/process"
        headers_request = { "Authorization": f"Bearer {token['access_token']}" }
        response = oauth.post(url_request, headers=headers_request, json=json_request)
        if response.status_code != 200:
            print(f"Error {response.status_code} para {lago} {fecha}")
            continue
        outfile = f"out/{lago}_{fecha}_multibanda_9b.tif"
        with open(outfile, "wb") as f:
            f.write(response.content)
        print(f"Guardado: {outfile}")

Guardado: out/amatitlan_2025-02-07_multibanda_9b.tif
Guardado: out/amatitlan_2025-02-10_multibanda_9b.tif
Guardado: out/amatitlan_2025-02-10_multibanda_9b.tif
Guardado: out/amatitlan_2025-02-25_multibanda_9b.tif
Guardado: out/amatitlan_2025-02-25_multibanda_9b.tif
Guardado: out/amatitlan_2025-02-27_multibanda_9b.tif
Guardado: out/amatitlan_2025-02-27_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-02_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-02_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-04_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-04_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-07_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-07_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-09_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-09_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-12_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-12_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-14_multibanda_9b.tif
Guardado: out/amatitlan_2025-03-14_multibanda_